In [1]:
import os
import pandas as pd
import numpy as np
import tqdm

In [2]:
class Exon:
    def __init__(self, gene, chrom, strand):
        self.gene = gene
        self.chrom = chrom
        self.strand = strand
        self.start = set()
        self.end = set()
        self.txL = 214748364700
        self.txR = -1
        pass

    def add_left(self, left):
        self.end.add(left)

    def add_right(self, right):
        self.start.add(right)
    
    def check_bound(self,L,R):
        self.txL =min(self.txL,L)
        self.txR = max(self.txR,R)
    
    def check_paralog(self,flag):
        self.paralog = flag

    def add_area(self, left, right):
        self.add_left(left)
        self.add_right(right)

    def format(self, s):
        return ','.join(str(i) for i in list(s))

In [3]:
junction_file = r'D:\CODE\BIO\dataset\gtex\GTEx_Analysis_2017-06-05_v8_STARv2.5.3a_junctions.gct'
sample_list = r'D:\CODE\BIO\dataset\gtex\GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt'
gene_db = r'D:\CODE\BIO\dataset\gtex\v8\mart_export.txt'
para_db = r'D:\CODE\BIO\dataset\gtex\v8\mart_export_paralog.txt'
output_path = r'D:\CODE\BIO\dataset\gtex\v8\processed'

## build sample list

In [3]:
sample_df = pd.read_csv(sample_list,delimiter='\t')
sample_df = sample_df[['SAMPID','SMTS','SMTSD']]
sample_df.head(20)
target_file = os.path.join(output_path,'sample.csv')
sample_df.to_csv(target_file,index=None)

## get tissue label

In [8]:
def name_decorate(name):
    res = str(name).replace(' ','')
    res = res.replace('-','_')
    return res

In [9]:
sample_file = os.path.join(output_path,'sample.csv')
sample_list = pd.read_csv(sample_file)
tissue_type = sample_list['SMTSD'].unique()
tissue_type.sort()
'''
for idx,name in enumerate(tissue_type):
    # print('{}:\'{}\','.format(idx,name))
    print('{}:\'{}\','.format(idx,name_decorate(name)))
    pass
'''
print(tissue_type)

0:'Adipose_Subcutaneous',
1:'Adipose_Visceral(Omentum)',
2:'AdrenalGland',
3:'Artery_Aorta',
4:'Artery_Coronary',
5:'Artery_Tibial',
6:'Bladder',
7:'Brain_Amygdala',
8:'Brain_Anteriorcingulatecortex(BA24)',
9:'Brain_Caudate(basalganglia)',
10:'Brain_CerebellarHemisphere',
11:'Brain_Cerebellum',
12:'Brain_Cortex',
13:'Brain_FrontalCortex(BA9)',
14:'Brain_Hippocampus',
15:'Brain_Hypothalamus',
16:'Brain_Nucleusaccumbens(basalganglia)',
17:'Brain_Putamen(basalganglia)',
18:'Brain_Spinalcord(cervicalc_1)',
19:'Brain_Substantianigra',
20:'Breast_MammaryTissue',
21:'Cells_Culturedfibroblasts',
22:'Cells_EBV_transformedlymphocytes',
23:'Cells_Leukemiacellline(CML)',
24:'Cervix_Ectocervix',
25:'Cervix_Endocervix',
26:'Colon_Sigmoid',
27:'Colon_Transverse',
28:'Esophagus_GastroesophagealJunction',
29:'Esophagus_Mucosa',
30:'Esophagus_Muscularis',
31:'FallopianTube',
32:'Heart_AtrialAppendage',
33:'Heart_LeftVentricle',
34:'Kidney_Cortex',
35:'Kidney_Medulla',
36:'Liver',
37:'Lung',
38:'MinorSal

## build junction list by chromosome

In [51]:
def split_junction_name(df,col):
    split_name = df['Name'].split(sep='_')
    df['chr'] = split_name[0]
    df['left'] = split_name[1]
    df['right'] = split_name[2]
    return df
def cut_description(df):
    # print(df['Description'])
    s = str(df['Description'])
    # print(s[0:s.find('.')])
    df['Description'] = s[0:s.find('.')]
    return df

In [4]:
read_chunks = pd.read_csv(junction_file,sep='\t',header=2,iterator=True,chunksize=128,low_memory=False)

In [33]:
def solve1():
    header = None
    cnt = 0
    for chunk in tqdm.tqdm(read_chunks):
        if header is None:
            header = chunk.columns
        # print(header)
        df = chunk.apply(split_junction_name,col='Name',axis=1)
        df = df.drop('Name',axis=1)
        order = ['chr','left','right'] + list(df.columns[:-3])
        # print(order)
        df = df[order]
        chr_list = df['chr'].unique()
        for chr in chr_list:
            target_csv = os.path.join(output_path,chr+'.csv')
            first = not os.path.exists(target_csv)
            if len(chr_list)>1:
                df[df['chr']==chr].to_csv(target_csv,mode='a',index=False,header=first)
            else:
                df.to_csv(target_csv,mode='a',index=False,header=first)
        first = False
    # df = read_chunks.__next__()
    # df = df.apply(split_junction_name,col='name',axis=1)
    # df.head()
    pass

32


## make paralog list

In [4]:
para_df = pd.read_csv(para_db,header=0,sep='\t')
para_df['paralogs'] = para_df['Human paralogue gene stable ID'].notnull()
para_df['paralogs'] = para_df['paralogs'].map({True:1,False:0})

In [5]:
para_df = para_df[['Gene stable ID','paralogs']]
para_df.columns = ['description','paralog']
para_df = para_df.drop_duplicates('description',keep='first')
para_df.head()

,description,paralog
0,ENSG00000210049,0
1,ENSG00000211459,0
2,ENSG00000210077,0
3,ENSG00000210082,0
4,ENSG00000209082,0


In [5]:
lock = False
if lock:
    solve1()


2795it [3:23:23,  4.37s/it]


In [6]:
chr_list= ['chr1','chr2','chr3','chr4','chr5','chr6',
    'chr7','chr8','chr9','chr10','chr11','chr12',
    'chr13','chr14','chr15','chr16','chr16','chr17','chr18',
    'chr19','chr20','chr21','chr22']

In [54]:
gene_df = pd.read_csv(gene_db,header=0,sep='\t')
# gene_df= gene_df.drop_duplicates('Transcript stable ID',keep='first')
gene_df = gene_df[['Gene stable ID','Gene name','Transcript start (bp)','Transcript end (bp)','Strand']]
gene_df['Description'] = gene_df['Gene stable ID']
# gene_df.head()
for chr in chr_list:
    fpath = os.path.join(output_path,chr+'.csv')
    df = pd.read_csv(fpath,header=0)
    df = df[['chr','left','right','Description']]
    df = df.apply(cut_description,axis=1)
    df = df.merge(gene_df,on='Description',how='inner')
    df = df[(df['left']>=df['Transcript start (bp)'])&(df['right']<=df['Transcript end (bp)'])]
    df = df.drop_duplicates(['left','right'],keep='first')
    foutpath = os.path.join(output_path,chr+'_after.csv')
    df = df[['left','right','Description','Gene name','Strand','Transcript start (bp)','Transcript end (bp)']]
    df.columns = ['left','right','description','gene','strand','txStart','txEnd']
    df.to_csv(foutpath,index=False)
    

## make dataset

In [7]:
import warnings
import pandas as pd

warnings.simplefilter(action="ignore",category=FutureWarning)

exon_dict = {}
for chrom_name in chr_list:
    fin = os.path.join(output_path,chrom_name+'_after.csv')
    chrom_file = pd.read_csv(fin)
    chrom_file = chrom_file.merge(para_df,on='description',how='inner')
    for idx, row in tqdm.tqdm(chrom_file.iterrows(), total=chrom_file.shape[0]):
        gene = row['description']
        strand = row['strand']
        left = row['left']
        right = row['right']
        if gene not in exon_dict:
            exon_dict[gene] = Exon(gene, chrom_name, strand)
        exon_dict[gene].add_area(left, right)
        exon_dict[gene].check_bound(row['txStart'],row['txEnd'])
        exon_dict[gene].check_paralog(row['paralog'])
    # break
df = pd.DataFrame(
columns=('gene', 'paralog','chrom', 'strand','txStart','txEnd', 'exonEnds', 'exonStarts'))
for idx, exon in tqdm.tqdm(enumerate(exon_dict.values()), total=len(exon_dict)):
    df = df.append(
        pd.Series({'gene': exon.gene,'paralog':exon.paralog, 'chrom': str(exon.chrom), 'strand': exon.strand,
                  'txStart':exon.txL, 'txEnd':exon.txR,
                  'exonEnds': exon.format(exon.end), 'exonStarts': exon.format(exon.start)}),
        ignore_index=True
    )
df.to_csv('new_gtex_dataset2.txt',sep='\t',index=False,header=False)

100%|██████████| 33165/33165 [05:28<00:00, 101.09it/s]
